In [73]:
import os
import openai
from dotenv import load_dotenv
from googleapiclient.discovery import build

# 環境変数の読み込み
load_dotenv()
openai.api_type = os.environ["API_TYPE"]
openai.api_base = os.environ["API_BASE"]
openai.api_version = os.environ["API_VERSION"]
openai.api_key = os.environ["API_KEY"]

engine = os.environ["API_ENGINE"]

google_cse_id = os.environ['GOOGLE_CSE_ID']
google_api_key = os.environ['GOOGLE_API_KEY']

In [74]:
import json


def get_search_result(keyword: str, date_number: int, date_unit="d"):
    """
    指定したキーワードの検索結果を取得する関数
    """
    service = build(
        "customsearch",
        "v1",
        developerKey=google_api_key,
    )
    result = (
        service.cse()
        .list(
            q=keyword,
            dateRestrict=f"{date_number}{date_unit}",
            cx=google_cse_id,
        )
        .execute()
    )

    # 検索結果のトップ3を入れるに取得
    result_list = []
    for i, item in enumerate(result["items"]):
        result_list.append({"title": f'{item["title"]}{item["snippet"]}'})
        if i >= 2:
            break
    return json.dumps({"result": result_list})

In [75]:
# googleAPIのテスト
ret = get_search_result("東京駅のイベント", 1, "m")
ret2 = json.loads(ret)
for n in ret2["result"]:
    print("-"*20)
    print(n["title"])

--------------------
東京キャラクターストリート特設 ｜ 東京駅一番街東京キャラクターストリート内の「いちばんプラザ」やワゴンショップなど期間限定ショップやイベントも開催中！ いつも楽しいワクワクを集めてみなさまのお越しを ...
--------------------
東京駅周辺・丸の内でおすすめのイベント｜レッツエンジョイ東京東京駅周辺・丸の内でおすすめのイベント · WHITE KITTE · Marunouchi Bright Christmas 2023 「Disney DREAMS & WISHES」 · 丸の内イルミネーション 2023.
--------------------
イベント&ニュース ｜ 東京駅一番街イベント&ニュース · New! · 「THE KING OF FIGHTERS XIII」 POPUP SHOP in 東京キャラクターストリート · ワーナー・ブラザース100周年記念ポップアップショップ · 薬屋の ...


In [76]:
func1 = {
    "name": "get_search_result",
    "description": "指定したキーワードの検索結果を取得する",
    "parameters": {
        "type": "object",
        "properties": {
            "keyword": {
                "type": "string",
                "description": "キーワード",
            },
            "date_number": {
                "type": "number",
                "description": "日付や時間の範囲",
            },
            "date_unit": {
                "type": "string",
                "enum": ["d", "h", "y"],
            },
        },
         "required": ["keyword"],
    },
}

functions=[func1]

In [77]:
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"

response = openai.ChatCompletion.create(
    engine=ENGINE,
    messages=[
        {"role": "user", "content": question},
    ],
    functions=functions,
    function_call="auto",
)

response

<OpenAIObject chat.completion id=chatcmpl-8ZXCfTuSXgWhDOkZHCmFSH7bLrjr8 at 0x1191d2450> JSON: {
  "id": "chatcmpl-8ZXCfTuSXgWhDOkZHCmFSH7bLrjr8",
  "object": "chat.completion",
  "created": 1703479805,
  "model": "gpt-35-turbo-16k",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "role": "assistant",
        "function_call": {
          "name": "get_search_result",
          "arguments": "{\n  \"keyword\": \"\u6771\u4eac\u99c5\u306e\u30a4\u30d9\u30f3\u30c8\",\n  \"da

In [78]:
# 言語モデルからの回答メッセージを取得
message = response["choices"][0]["message"]

# モデルが関数呼び出しと判断した
if message.get("function_call"):
    # 関数名の取得
    function_name = message["function_call"]["name"]
    print(f"関数目: {function_name}")
    # 引数の取得
    arguments = json.loads(message["function_call"]["arguments"])
    print(f"引数: {arguments}")

    # 関数で実行する関数を判断
    if function_name == "get_search_result":
        function_response = get_search_result(
            keyword=arguments.get("keyword"),
            date_number=arguments.get("date_number"),
            date_unit=arguments.get("date_unit"),
        )
    print(f"関数の実行結果: {function_response}")

    # 関数の実行結果をmessagesに加えて言語モデルを呼び出し
    response2 = openai.ChatCompletion.create(
        engine=ENGINE,
        messages=[
            {
                "role": "user",
                "content": question,
            },
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )

    # 言語モデルからの回答を出力
    print(response2.choices[0]["message"]["content"].strip())

else:
    # 関数呼び出しでなければ単に言語モデルからの回答を出力
    print(response.choices[0]["message"]["content"].strip())

関数目: get_search_result
引数: {'keyword': '東京駅のイベント', 'date_number': 1, 'date_unit': 'M'}
関数の実行結果: {"result": [{"title": "\u6771\u4eac\u30ad\u30e3\u30e9\u30af\u30bf\u30fc\u30b9\u30c8\u30ea\u30fc\u30c8\u7279\u8a2d \uff5c \u6771\u4eac\u99c5\u4e00\u756a\u8857\u6771\u4eac\u30ad\u30e3\u30e9\u30af\u30bf\u30fc\u30b9\u30c8\u30ea\u30fc\u30c8\u5185\u306e\u300c\u3044\u3061\u3070\u3093\u30d7\u30e9\u30b6\u300d\u3084\u30ef\u30b4\u30f3\u30b7\u30e7\u30c3\u30d7\u306a\u3069\u671f\u9593\u9650\u5b9a\u30b7\u30e7\u30c3\u30d7\u3084\u30a4\u30d9\u30f3\u30c8\u3082\u958b\u50ac\u4e2d\uff01 \u3044\u3064\u3082\u697d\u3057\u3044\u30ef\u30af\u30ef\u30af\u3092\u96c6\u3081\u3066\u307f\u306a\u3055\u307e\u306e\u304a\u8d8a\u3057\u3092\u00a0..."}, {"title": "\u6771\u4eac\u99c5\u5468\u8fba\u30fb\u4e38\u306e\u5185\u3067\u304a\u3059\u3059\u3081\u306e\u30a4\u30d9\u30f3\u30c8\uff5c\u30ec\u30c3\u30c4\u30a8\u30f3\u30b8\u30e7\u30a4\u6771\u4eac\u6771\u4eac\u99c5\u5468\u8fba\u30fb\u4e38\u306e\u5185\u3067\u304a\u3059\u3059\u3081\u306e\u

In [79]:
import json
import requests
from bs4 import BeautifulSoup

def get_news(keyword, date_number, date_unit="d"):
    """
    指定したキーワードのニュースを取得する関数
    """
    # URLを指定
    url = f'https://news.google.com/search?q={keyword}when:{date_number}{date_unit}&hl=ja&gl=JP&ceid=JP:ja'
    # Requestを利用してwebページを取得する
    print(f"URL: {url}")
    response = requests.get(url)
    html = response.text

    # BeautifulSoupを利用してWebページを解析する
    soup = BeautifulSoup(html, 'html.parser')

    # soup.selectを利用して、ニュースのタイトルを取得する
    elements = soup.select('article div.m5k28 div.B6pJDd div a') # yDmH0d > c-wiz > div > main > div.UW0SDc > c-wiz > c-wiz:nth-child(1) > c-wiz > article > div.m5k28 > div.B6pJDd > div > a
    # print(elements[0])

    # ニュースのtop3を配列に取得
    news = []
    for i, element in enumerate(elements):
        news.append({"title": element.getText()})
        if i >=2:
            break

    return json.dumps({"news": news})

ret = get_news("クリスマス", 5)
ret2 = json.loads(ret)
# print(ret2["news"])
for n in ret2["news"]:
    print("-"*20)
    print(n["title"])

URL: https://news.google.com/search?q=クリスマスwhen:5d&hl=ja&gl=JP&ceid=JP:ja
--------------------
クリスマスケーキ崩れた状態で届く 高島屋“原因究明急ぐ”
--------------------
「スカスカおせち思い出した」高島屋“グチャグチャ”クリスマス ...
--------------------
クリスマスケーキ「ぐちゃぐちゃ」 高島屋が謝罪 被害訴え370件 ...


In [80]:
func2 = {
    "name": "get_news",
    "description": "指定したキーワードのニュースを取得する",
    "parameters": {
        "type": "object",
        "properties": {
            "keyword": {
                "type": "string",
                "description": "キーワード",
            },
            "date_number": {
                "type": "number",
                "description": "日付や時間の範囲",
            },
            "date_unit": {
                "type": "string",
                "enum": ["d", "h", "y"],
            },
        },
        "required": ["keyword"],
    }
}

functions.append(func2)

In [81]:
# question = "クリスマスに関する5日以内のニュースを教えてください"
question = "クリスマスに関する１週間以内のイベントを教えてください"

response = openai.ChatCompletion.create(
    engine=ENGINE,
    messages=[
        {"role": "user", "content": question},
    ],
    functions=functions,
    function_call="auto",
)

response

<OpenAIObject chat.completion id=chatcmpl-8ZXClzEnrBz9BWLxzAwNcdyUiXoq4 at 0x11f52f950> JSON: {
  "id": "chatcmpl-8ZXClzEnrBz9BWLxzAwNcdyUiXoq4",
  "object": "chat.completion",
  "created": 1703479811,
  "model": "gpt-35-turbo-16k",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "role": "assistant",
        "function_call": {
          "name": "get_search_result",
          "arguments": "{\n  \"keyword\": \"\u30af\u30ea\u30b9\u30de\u30b9 \u30a4\u30d9\u30f3\u30c8\",\

In [82]:
# 言語モデルからの回答メッセージを取得
message = response["choices"][0]["message"]

# モデルが関数呼び出しと判断した
if message.get("function_call"):
    # 関数名の取得
    function_name = message["function_call"]["name"]
    print(f"関数目: {function_name}")
    # 引数の取得
    arguments = json.loads(message["function_call"]["arguments"])
    print(f"引数: {arguments}")

    # 関数で実行する関数を判断
    if function_name == "get_search_result":
        function_response = get_search_result(
            keyword=arguments.get("keyword"),
            date_number=arguments.get("date_number"),
            date_unit=arguments.get("date_unit"),
        )
    elif function_name == "get_news":
        function_response = get_news(
            keyword=arguments.get("keyword"),
            date_number=arguments.get("date_number"),
            date_unit=arguments.get("date_unit"),
        )
    print(f"関数の実行結果: {function_response}")

    # 関数の実行結果をmessagesに加えて言語モデルを呼び出し
    response2 = openai.ChatCompletion.create(
        engine=ENGINE,
        messages=[
            {
                "role": "user",
                "content": question,
            },
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )

    # 言語モデルからの回答を出力
    print(response2.choices[0]["message"]["content"].strip())

else:
    # 関数呼び出しでなければ単に言語モデルからの回答を出力
    print(response.choices[0]["message"]["content"].strip())

関数目: get_search_result
引数: {'keyword': 'クリスマス イベント', 'date_number': 7, 'date_unit': 'd'}
関数の実行結果: {"result": [{"title": "\u6771\u4eac\u90fd\u5185\u306e\u30af\u30ea\u30b9\u30de\u30b9\u30a4\u30d9\u30f3\u30c82023\uff5c\u30ec\u30c3\u30c4\u30a8\u30f3\u30b8\u30e7\u30a4\u6771\u4eac\u6771\u4eac\u30af\u30ea\u30b9\u30de\u30b9\u30de\u30fc\u30b1\u30c3\u30c82023 ... \u65e5\u672c\u6700\u5927\u7d1a\u306e\u30a4\u30d9\u30f3\u30c8\u3001\u660e\u6cbb\u795e\u5bae\u5916\u82d1\u3067\u958b\u50ac\u6c7a\u5b9a\uff01 \u5f53\u30a4\u30d9\u30f3\u30c8\u306f\u6628\u5e74\u65e5\u6bd4\u8c37\u516c\u5712\u306b\u306625\u4e07\u4eba\u306e\u6765\u5834\u8005\u3092\u96c6\u3081\u3001\u51fa\u5e97\u898f\u6a21\u3082\u76f8\u307e\u3063\u3066\u540d\u5b9f\u3068\u3082\u306b\u00a0..."}, {"title": "\u30ca\u30a4\u30c8\u30c9\u30ec\u30b911\u7740\u30bb\u30c3\u30c8 \u30d0\u30e9\u58f2\u308a \u30af\u30ea\u30b9\u30de\u30b9 \u30a4\u30d9\u30f3\u30c8 - \u5bb6\u96fb\u30ca\u30a4\u30c8\u30c9\u30ec\u30b911\u7740\u30bb\u30c3\u30c8 \u30d0\u30e9\u58f2\u308a